In [1]:
# 風傳媒
# 爬蟲套件
import requests
from bs4 import BeautifulSoup
import json
from fake_useragent import UserAgent
# 資料整理類套件
import pandas as pd
import re
import time
import random
import datetime
from tomorrow import threads
from pymongo import MongoClient
from datetime import datetime
import pymongo

In [2]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [3]:
def GetAllLinks():
    '''最後一頁是2904'''
    links = []
    page = 1
    while True:
        url = "https://www.storm.mg/category/118/"+str(page)
        res = requests.get(url,headers=headers)
        soup = BeautifulSoup(res.text)
        category_card = soup.find_all("div",class_="category_card card_thumbs_left")
        if len(category_card) == 0:
            break
        if page > 50:
            break
        for i in category_card:
            links.append(i.find("a",class_="card_link link_img").get("href"))
        page += 1
    return links

In [4]:
def GetStorm(response):
    '''風傳媒內容爬取'''
    soup = BeautifulSoup(response.text)
    Title = soup.find("h1",id="article_title").text
    Url = soup.find("link",rel="canonical").get("href")
    
    Author = soup.find("div",id="author_block").find("span",class_="info_author").text
    CreateTime = soup.find("div",id="time_pop_block").find("span",class_="info_inner_content").text
    CreateTime = datetime.strptime(CreateTime,'%Y-%m-%d %H:%M')
    list_ = []
    for i in soup.find("div",id="article_inner_wrapper").find_all("p"):
        list_.append(i.text)
    Content = "".join(list_)
    Tag = []
    for i in soup.find("div",id="tags_wrapper").find_all("a",class_="tag"):
        Tag.append(i.text)
        
    ndf = pd.DataFrame(data = [{"Source":"News",
                                "Site":"風傳媒",
                                "Url":Url,
                                "Title":Title,
                                "Author":Author,
                                "CreateTime":CreateTime,
                                "Content":Content,
                                "Tag":Tag}],
                      columns = ['Source', 'Site', 'Url', 'CreateTime', 'Author', 'Title', 'Content','Tag'])
    return ndf


In [5]:
# testurl = 'https://www.storm.mg/article/1969753'
# response = requests.get(testurl,headers=headers)
# GetStorm(response)

In [6]:
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [7]:
def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = GetStorm(response)
            allData.append(ndf)
        except:
            pass
    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df

In [8]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_news  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")


In [9]:
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
        

In [10]:
def dataframe_to_mongo_political(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
    

In [11]:
links = GetAllLinks()
df = CrawlAllNews(links)
dataframe_to_mongo_political(df)

There are 650 links in pages.
There are 650 News in DataFrame.
